# **Veri ve modül içe aktarımı**

In [31]:
import pandas as pd
pd.set_option('display.max_columns', None)

test_data=pd.read_csv('test.csv')
train_data=pd.read_csv('train.csv')
test_label=pd.read_csv('sample_submission.csv')



# VERİ SETİ ÖN İŞLEME

In [32]:
X_train = train_data.drop('SalePrice', axis=1) 
y_train = train_data['SalePrice']
X_test = test_data.copy()  
y_test = test_label['SalePrice'] 


missing_percentages = X_train.isnull().mean() * 100 
high_missing_cols = missing_percentages[missing_percentages > 80].index


X_train.drop(columns=high_missing_cols, inplace=True)  
X_test.drop(columns=high_missing_cols, inplace=True)


object_cols = X_train.select_dtypes(include=['object']).columns
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns


modes = X_train[object_cols].mode().iloc[0]
X_train[object_cols] = X_train[object_cols].fillna(modes)
X_test[object_cols] = X_test[object_cols].fillna(modes)


medians = X_train[num_cols].median()
X_train[num_cols] = X_train[num_cols].fillna(medians)
X_test[num_cols] = X_test[num_cols].fillna(medians)

X_train.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Gd,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,BrkFace,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,BrkFace,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal


# Veri Encode İşlemi

In [33]:
from sklearn.preprocessing import LabelEncoder 

label_enc=LabelEncoder()

for col in object_cols:
    if col in X_train.columns: 
        X_train[col] = label_enc.fit_transform(X_train[col])
        X_test[col] = label_enc.transform(X_test[col])


# **Multiple Linear Regression** 

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [35]:
model = LinearRegression()
model.fit(X_train, y_train) # encoded veri ile deniyoruz
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


print("MLR Performans Metrikleri:")
print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R^2: {r2:.2f}\n")



MLR Performans Metrikleri:
MSE: 523319879.77
MAE: 15785.37
R^2: 0.91



# **Karar Ağacı** 

In [47]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(
    max_depth=100,
    min_samples_split=5,
    min_samples_leaf=5,
    random_state=42
)
tree_reg.fit(X_train, y_train)
y_tree_pred = tree_reg.predict(X_test)

mse = mean_squared_error(y_test, y_tree_pred)
mae = mean_absolute_error(y_test, y_tree_pred)
r2 = r2_score(y_test, y_tree_pred)


print("Karar Ağacı Performans Metrikleri:")
print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R^2: {r2:.2f}\n")

Karar Ağacı Performans Metrikleri:
MSE: 901440330.49
MAE: 19054.27
R^2: 0.85



# **Random Forest** Başlamadı

In [48]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=25,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

rf_model.fit(X_train, y_train)
y_rf_pred = rf_model.predict(X_test)

mse = mean_squared_error(y_test, y_rf_pred)
mae = mean_absolute_error(y_test, y_rf_pred)
r2 = r2_score(y_test, y_rf_pred)

print("Random Forest Performans Metrikleri:")
print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R^2: {r2:.2f}\n")

Random Forest Performans Metrikleri:
MSE: 246242566.72
MAE: 9766.94
R^2: 0.96

